In [3]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

#tf.set_random_seed(1)

%run Dataset.ipynb
%run ImageGenerator.ipynb

IMAGE_HEIGHT = 100
IMAGE_WEIGHT = 100
BATCH_SIZE = 10


data_dir = os.getcwd()
X_path = data_dir + '/OtherPerson'
Y_path = data_dir + '/OtherPerson' #""#autoencoder##data_dir + '/y'

log_dir = data_dir + '/Tensorboard/' + 'Temp/'

image_generator = ImageGenerator(IMAGE_HEIGHT, IMAGE_WEIGHT)
data_set = Dataset(X_path, Y_path, BATCH_SIZE, image_generator, True)

#CHECK
data_set.current_batch_index = 3
check_images, _ = np.array(data_set.get_next_batch(32))
check_image = check_images[4]
print(check_images.shape)
plt.imshow(check_image)

(32, 100, 100, 3)


In [4]:
%run Autoencoder.ipynb

tf.reset_default_graph() 

autoencoder = Autoencoder()
autoencoder.build_model()
autoencoder.summary()

print(autoencoder.input)
print(autoencoder.output)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.


ValueError: Variable conv4/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-4-518777160521>", line 18, in build_model
    x = conv2d(x, filters=24, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', name='conv4')
  File "<ipython-input-4-6da563211d59>", line 6, in <module>
    autoencoder.build_model()
  File "/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [1]:
loss = tf.reduce_mean(tf.square(autoencoder.output - autoencoder.input))  # claculate the mean square error loss
tf.summary.scalar('loss', loss)

learning_rate = 0.0001
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

sess = tf.Session()
saver = tf.train.Saver()

sess_file_path = log_dir + 'model.ckpt'
# saver.restore(sess, sess_file_path)

file_path = log_dir + '1'
writer = tf.summary.FileWriter(file_path)
writer.add_graph(sess.graph)
print(file_path)

sess.run(tf.global_variables_initializer())

NameError: name 'tf' is not defined

In [2]:
Y = tf.placeholder(dtype=tf.float32, shape=(None, IMAGE_HEIGHT, IMAGE_WEIGHT, 3))
epochs = 10

print_every_ep_index = 1
print_every_b_index = 4

saver = tf.train.Saver()

merged_summary = tf.summary.merge_all()
for ep_index in range(epochs):
    #Reset set to 0
    data_set.reset_index()
    
    if ep_index % print_every_ep_index == 0:
        print('EPOCH #{}'.format(ep_index))
        
    for batch_step in range(data_set.batches_count):
        train_images, _ = np.array(data_set.get_next_batch(BATCH_SIZE))
        gr, batch_loss = sess.run([optimizer, loss], feed_dict = { autoencoder.input : train_images })

        if data_set.current_batch_index % print_every_b_index == 0:
            s = sess.run(merged_summary, feed_dict = { autoencoder.input : train_images })
            writer.add_summary(s, ep_index + batch_step)
            
#             saver.save(sess, save_file_path)
            print('   Loss {}'.format(batch_loss))

NameError: name 'tf' is not defined

In [ ]:
#Test autoencoder
test_batch_size = 20
data_set.reset_index()
train_images, _ = np.array(data_set.get_next_batch(test_batch_size))
images = sess.run(autoencoder.output, feed_dict = { autoencoder.input : train_images })
print(images.shape)

test_image_index = 6
plt.imshow(train_images[test_image_index])

In [ ]:
plt.imshow(images[test_image_index])